##### Aim Of the Project is to get the product name, new price, old price and number of reviews of a product from Amazon product page
##### This Project illustrate how to Navigate to next page from current page Dynamically (Without using url for next page) and i have scraped product information of 20 pages.

#### First Page URL (Headset page) - https://www.amazon.com/s?k=headphones&crid=T7DXZT5BZT92&qid=1637732331&sprefix=head%2Caps%2C203

In [1]:
!pip install requests --quiet
!pip install bs4 --quiet

### Loading the Required packages

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Adding user-agent and this can be found in https://httpbin.org/get

In [3]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 Edg/96.0.1054.29", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

#### First Page URL and Base URL Respectively 

In [4]:
base_url ="https://www.amazon.com/s?k=headphones&crid=T7DXZT5BZT92&qid=1637732331&sprefix=head%2Caps%2C203"
base = "https://www.amazon.com"

#### Function to get the each page content passing different URL 

In [5]:
def get_doc(url):
    response = requests.get(url,headers=headers)
    print(response.status_code)
    if response.ok:
        with open("amazon_html_page.html", "a",encoding='utf8') as file:
            file.write(response.text)
        doc = BeautifulSoup(response.text, "html.parser")
        return doc

#### Function to get different Page URL Dynamically

In [6]:
def getnextpage(doc):
    page = doc.find("ul",class_ = "a-pagination")
    if not page.find("li", class_ = "a-disabled a-last"):
        url = base+str(page.find("li",class_="a-last").find("a")["href"])
        print(url)
        return url
    else:
        return

#### Function to get required information from each page

In [18]:
def web_scrap_data(content):
    h2_tag = content.find("h2")
    product_name = h2_tag.text.strip()
    price = content.find_all("span", class_="a-price")
    if len(price) > 0:
        new_price = price[0].find("span",class_ ="a-offscreen").text.strip()
    else:
        new_price = "Not Specified"
    if len(price) > 1:
        old_price = price[1].find("span",class_ ="a-offscreen").text.strip()
    else:
        old_price = "Not Specified"
    n_reviews = content.find("span", class_="a-size-base")
    try:
        n_reviews = n_reviews.text.strip()
    except:
        n_reviews = "0"
#     n_reviews = n_reviews.string.replace(",","")
#     n_reviews = int(n_reviews)
    return {
        "Product_name":product_name,
        "New_price": new_price,
        "Old_price":old_price,
        "Number_of_reviews":n_reviews
    }

#### Function to get the container which have all the required information

In [8]:
def get_content(doc):
    return (doc.find_all("div", class_="s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"))

In [19]:
url = base_url
link_list = []
count = 0
while True:
    doc = get_doc(url)
    page_content = get_content(doc)
    amazon_headset = [web_scrap_data(content) for content in page_content]
    if count ==0:
        product = pd.DataFrame.from_dict(amazon_headset)
    else:
        product = pd.concat((product,pd.DataFrame.from_dict(amazon_headset)),axis=0)
    url = getnextpage(doc)
    if not url:
        break
    else:
        count+=1

200
https://www.amazon.com/s?k=headphones&page=2&crid=T7DXZT5BZT92&qid=1637997263&sprefix=head%2Caps%2C203&ref=sr_pg_1
200
https://www.amazon.com/s?k=headphones&page=3&crid=T7DXZT5BZT92&qid=1637997265&sprefix=head%2Caps%2C203&ref=sr_pg_2
200
https://www.amazon.com/s?k=headphones&page=4&crid=T7DXZT5BZT92&qid=1637997267&sprefix=head%2Caps%2C203&ref=sr_pg_3
200
https://www.amazon.com/s?k=headphones&page=5&crid=T7DXZT5BZT92&qid=1637997268&sprefix=head%2Caps%2C203&ref=sr_pg_4
200
https://www.amazon.com/s?k=headphones&page=6&crid=T7DXZT5BZT92&qid=1637997270&sprefix=head%2Caps%2C203&ref=sr_pg_5
200
https://www.amazon.com/s?k=headphones&page=7&crid=T7DXZT5BZT92&qid=1637997272&sprefix=head%2Caps%2C203&ref=sr_pg_6
200
https://www.amazon.com/s?k=headphones&page=8&crid=T7DXZT5BZT92&qid=1637997273&sprefix=head%2Caps%2C203&ref=sr_pg_7
200
https://www.amazon.com/s?k=headphones&page=9&crid=T7DXZT5BZT92&qid=1637997275&sprefix=head%2Caps%2C203&ref=sr_pg_8
200
https://www.amazon.com/s?k=headphones&page=1

In [21]:
product.to_csv("Product_Details.csv",index=False)

In [20]:
product

,Product_name,New_price,Old_price,Number_of_reviews
0,JBL Live 460NC - Wireless On-Ear Noise Cancell...,$64.95,$129.95,105
1,Sony WH-1000XM4 Wireless Industry Leading Nois...,$248.00,$348.00,"24,263"
2,JBL TUNE 500 - Wired On-Ear Headphones - Black,$19.95,$29.95,"1,995"
3,"Sony ZX Series Wired On-Ear Headphones, Black ...",$9.99,Not Specified,"70,626"
4,Sony Wireless Headphones WH-CH510: Wireless Bl...,$38.00,$59.99,"16,083"
...,...,...,...,...
13,YouthWhisper Bone Conduction Headphones Blueto...,$54.99,Not Specified,927
14,Sony WH1000XM3 Noise Cancelling Headphones : W...,Not Specified,Not Specified,"20,373"
15,Jabra Evolve2 85 UC Wireless Headphones with L...,$499.00,Not Specified,527
0,"Bluetooth Beanie Hat, Gifts for Men Women Chri...",$21.99,Not Specified,433
